In [164]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [165]:
from astropy import units as u
from astropy.coordinates import EarthLocation, SkyCoord
from astropy.time import Time

from astroplan.plots import plot_airmass, plot_sky, plot_sky_24hr, plot_finder_image
from matplotlib import pyplot as plt
%matplotlib inline

from astroplan import Observer

from pocs.scheduler.dispatch import Scheduler
from pocs.scheduler.constraint import Duration, MoonAvoidance
from pocs.utils.config import load_config
from pocs.utils import listify

from astropy import coordinates
import astropy.units as u
from astroquery.simbad import Simbad


config = load_config()

In [166]:
# For VOparser

from __future__ import print_function
import copy
import voeventparse
from voeventparse.tests.resources.datapaths import swift_bat_grb_pos_v2

with open(swift_bat_grb_pos_v2, 'rb') as f:
    v = voeventparse.load(f)

In [167]:
print("Ivorn:", v.attrib['ivorn'])
print("Role:", v.attrib['role'])
print( "AuthorIVORN:", v.Who.AuthorIVORN)
print( "Short name:", v.Who.Author.shortName)
print( "Contact:", v.Who.Author.contactEmail)

Ivorn: ivo://nasa.gsfc.gcn/SWIFT#BAT_GRB_Pos_532871-729
Role: observation
AuthorIVORN: ivo://nasa.gsfc.tan/gcn
Short name: VO-GCN
Contact: scott.barthelmy@nasa.gov


In [168]:
c = voeventparse.pull_astro_coords(v)
t = voeventparse.pull_isotime(v)
print( "Coords:", c)
print(t)

Coords: Position2D(ra=74.7412, dec=-9.3137, err=0.05, units='deg', system='UTC-FK5-GEO')
2012-09-07 00:24:23.080000+00:00


In [169]:
# returns string in desired format for POCS - converts decimal VO value to DegMinSec format
# other method converts time, so that it does not have as many dec places - this was an issue when pocs 
# ran the sunrise/ sunset time.

def convert_time(tim):
    time = str(tim)
    return time[0:19]

def convert_to_deg(num, typ):
    _h = int(num)
    _m=abs((num-_h)*60)
    _min = abs(int(_m))
    _s=abs(round((_m-_min)*60,3))
    
    hrs = str(_h)
    if typ == 'h':
        _h = int(_h*24/100)
        hrs = str(_h)
        
        if _h >= 0:
            hrs = '+'+hrs

    result = str(_h)+typ+str(_min)+"m"+str(_s)+"s"
    
    return result

def parse_err(err):
    return err*60* u.arcminute
    

In [170]:
ra = convert_to_deg(c[0], "h") #<- RA has first unit of hours, so we give the method the first unit in the string
dec = convert_to_deg(c[1],"d")

time = convert_time(t)

coords = [ra, dec]
print(coords) #<- checking that it works
print(time)

['17h44m28.32s', '-9d18m49.32s']
2012-09-07 00:24:23


In [187]:
# Get a location
loc = config['location']
location = EarthLocation(lon=loc['longitude'], lat=loc['latitude'], height=loc['elevation'])

# Get an observer at the location
observer = Observer(location=location)

# Get the night hours (you can use observer.tonight())
#times = observer.tonight(horizon=-18 * u.degree)
#start_of_night = times[0]
#end_of_night = times[1]
start_of_night = Time('2016-05-13 10:00:00')
end_of_night = Time('2016-05-13 16:00:00')

In [191]:
# Creating new config list to store our observation

new_confi = {'name': 'ToO', 'position': coords[0]+" "+coords[1], 'exp_time': '10', 'priority': '1000'}

In [192]:
# Here I want to create a new observation
simple_fields_file = config['directories']['targets'] + '/simple.yaml'

# Simple constraint to maximize duration above a certain altitude
constraints = [MoonAvoidance(), Duration(30 * u.deg)]

# Create a scheduler
scheduler = Scheduler(observer, fields_file=simple_fields_file, constraints=constraints)

scheduler.add_observation(new_confi)

In [193]:
# Let's see if it gets sent to the telescope!

# Get the best observations for the given time
# show_all returns entire list along with merit
best = scheduler.get_observation(time=start_of_night, show_all=True)

for b in listify(best):
    print("Field: {:12s} Merit: {}".format(b[0], b[1]))



2016-11-28 11:39:04 aaolxo panoptes[30534] INFO Setting new observation to ToO: 10.0 s exposures in blocks of 10, minimum 60, priority 1000


{'Kepler 1100': 127.65584746279617, 'Tres 3': 102.61256279716281, 'ToO': 1002.7052153706207}
Field: ToO          Merit: 1002.7052153706207
Field: Kepler 1100  Merit: 127.65584746279617
Field: Tres 3       Merit: 102.61256279716281


In [ ]:
# Trying to find referenec objects
r = parse_err(c[2])
c = coordinates.SkyCoord(new_confi['position'])

In [175]:

found = True

try: 
    all_objects_in_region = Simbad.query_region(c, radius = 100*r)
except:
    found = False

print(all_objects_in_region)

2016-11-28 11:21:51 aaolxo requests.packages.urllib3.connectionpool[30534] INFO Starting new HTTP connection (1): simbad.u-strasbg.fr


         MAIN_ID                RA      ... COO_WAVELENGTH     COO_BIBCODE    
                             "h:m:s"    ...                                   
------------------------- ------------- ... -------------- -------------------
  2MASX J17444776-0918057  17 44 47.769 ...              I 2006AJ....131.1163S
ZOA J174402.614-092145.97   17 44 02.61 ...              N 2014MNRAS.443...41W
          HIPASS J1743-09    17 43 58.8 ...              R 2005MNRAS.361...34D
          IRAS 17413-0911    17 44 07.6 ...              F 1988NASAR1190....1B
           TYC 5661-646-1  17 45 06.232 ...              O 2000A&A...355L..27H
  6dFGS gJ174430.9-090903  17 44 30.879 ...              I 2006AJ....131.1163S
                HD 161036 17 43 41.9623 ...                1998A&A...335L..65H
    1RXS J174336.5-092243  17 43 36.497 ...                                   
             LEDA  200706    17 43 29.1 ...                1996PASJ...48..679R
          IRAS 17407-0914   17 43 28.50 ...         

In [ ]:
if found == True:
    all_objects_in_region.pprint(show_unit=True, max_width=120, max_lines=5) # <- print discovered objects

In [ ]:
# Next we want to access images of the objects found